In [4]:
import platform
import qiskit
import qiskit_aer
print(f"Python version: {platform.python_version()} ({platform.python_version_tuple()[0]}.{platform.python_version_tuple()[1]})")
print (f"NumPy Version: {np.__version__}")
print(f"Qiskit Version: {qiskit.__version__}")
print(f"Qiskit Aer Version: {qiskit_aer.__version__}")

Python version: 3.10.19 (3.10)
NumPy Version: 2.2.6
Qiskit Version: 1.2.4
Qiskit Aer Version: 0.15.1


In [3]:

import math
import numpy as np
from fractions import Fraction
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit.circuit.library import QFT
from qiskit_aer import Aer


In [5]:
def iqft(n: int):
    return QFT(num_qubits=n, inverse=True, do_swaps=True, name="IQFT").to_instruction()

Purpose:
    Builds an Inverse Quantum Fourier Circuit on n Qubits
    
IQFT undoes the QFT and the conrolled modular multi to extract the periods in the counting registers

In [6]:

def continued_fraction_period(phase_float: float, N: int):
    frac = Fraction(phase_float).limit_denominator(N)
    return frac.denominator if frac.denominator > 0 else None

Purpose: 
    Estimates the r period of Mod Expo. 
    
    Takes the measured phase, float point between 0 and 1, and uses fraction approximation to find a fraction whos GCD corresponds to that found r

In [7]:
def make_modmul_gate(a_pow: int, N: int, n_mod: int):
    dim = 1 << n_mod
    if math.gcd(a_pow, N) != 1:
        raise ValueError(f"a_pow={a_pow} not invertible mod N")

    U = np.zeros((dim, dim), dtype=complex)
    for x in range(dim):
        y = (a_pow * x) % N if x < N else x
        U[y, x] = 1.0

    qc = QuantumCircuit(n_mod, name=f"U_mul_{a_pow}_mod_{N}")
    qc.unitary(U, range(n_mod))
    return qc.to_gate()


Purpose:
    Creates a unitary matrix that multiplies each number by a_pow (mod N).
    
    It builds a permutation matrix that represents the mod multi
    
    Matrix Converted into Qiskit gate

In [8]:
def make_controlled_modexp(a: int, N: int, power: int, n_mod: int):
    a_pow = pow(a, 2 ** power, N)
    base_gate = make_modmul_gate(a_pow, N, n_mod)
    return base_gate.control(1)

Purpose:
    create a controlled version of Mod Expo
    
    1. Computs a_pow = a^(2^power) mod N, builds the gate from make_modmul_gate then adding a controlled qubit
    2. Each counting qubit controls a different power, Shors need this for phase estimation

In [9]:
def build_period_finder(a: int, N: int):
    n_mod = math.ceil(math.log2(N))
    n_count = 2 * n_mod

    qr_count = QuantumRegister(n_count, "c")
    qr_mod = QuantumRegister(n_mod, "m")
    qc = QuantumCircuit(qr_count, qr_mod, name=f"QPE_a{a}_N{N}")

    qc.h(qr_count)
    qc.x(qr_mod[0])

    for j in range(n_count):
        cu = make_controlled_modexp(a, N, j, n_mod)
        qc.append(cu, [qr_count[j]] + list(qr_mod))

    qc.append(iqft(n_count), qr_count)
    return qc, n_count, n_mod

Purpose: 
    Constructs the full quantum pd-finding circuit
    
    1. It creates two registers, counting (superposition initialized) and modular (initialized to |1>) register.
    2. Appends controlled mod expo gates
    3. Applies IQFT to extract phase info

In [10]:
def run_qpe_statevector(a: int, N: int):
    qc, n_count, n_mod = build_period_finder(a, N)
    sim = Aer.get_backend("statevector_simulator")

    tqc = transpile(qc, sim, optimization_level=1)
    result = sim.run(tqc).result()

    psi = np.asarray(result.data(0)["statevector"])

    total = n_count + n_mod
    psi = psi.reshape([2] * total)

    probs = np.sum(np.abs(psi) ** 2, axis=tuple(range(n_count, total)))

    out = {}
    for k in range(1 << n_count):
        p = float(probs.flat[k])
        if p > 1e-12:
            out[format(k, f"0{n_count}b")] = p
    return out, n_count

Purpose:
    Simulates the above circuit using Qiskits statevector sim
    
    1. It builds and transpiles the QPE circuit 
    2. Runs it on Aer statevector Backend, converting the statevector into a numpy array (avoids warnings)
    3. Sums probabilities over mod qubits to get probabilites of each counting bitstring
    4. returns a dictionary

Highest probability estimates the period

In [11]:
def shor_factor(N: int, max_attempts: int = 12):
    if N % 2 == 0:
        return 2, N // 2, {"method": "trivial even"}

    for attempt in range(1, max_attempts + 1):
        a = np.random.randint(2, N)
        g = math.gcd(a, N)
        print(f"\n Attempt {attempt}: a = {a} (gcd={g})")

        if 1 < g < N:
            print(f"  -> classical gcd factor: {g}")
            return g, N // g, {"method": "classical gcd", "a": int(a)}

        probs, n_count = run_qpe_statevector(a, N)
        bitstr, pmax = max(probs.items(), key=lambda kv: kv[1])
        k = int(bitstr, 2)
        phase = k / (2 ** n_count)
        r = continued_fraction_period(phase, N)
        print(f"  top k={bitstr} p={pmax:.4f} → phase={phase:.6f} → r={r}")

        if r is None or r % 2 != 0:
            print("   invalid r; retrying...")
            continue
        if pow(a, r, N) != 1:
            print("   a^r != 1 mod N; retrying...")
            continue

        x = pow(a, r // 2, N)
        p = math.gcd(x - 1, N)
        q = math.gcd(x + 1, N)
        if 1 < p < N and 1 < q < N and p * q == N:
            print(f" SUCCESS: {N} = {p} × {q} [quantum r={r}]")
            return p, q, {"method": "shor_qpe", "a": int(a), "r": int(r)}
        else:
            print("  derived p,q invalid; retrying...")

    print("FAILED: Quantum Phase Estimation did not find valid period.")
    return None, None, {"method": "failed"}

Purpose: 
    Runs shors algorithm on int N combining classical and quantum

    1. If N is even -> trivial factor as 2 * (N/2)
    2. Picks random a and checks gcd(a, N), classical
    3. If gcd(a, N) = 1, run QPE for order r
    4. Verify r, if its even and valid, checks a^r ≡ 1 mod N
    5. Computes nontrivial factors

In [12]:
def run_batch(nums):
    results = {}
    for N in nums:
        print("\n========================================")
        print(f"Factoring N = {N}")
        print("========================================")
        p, q, meta = shor_factor(N)
        results[N] = (p, q, meta)
        print(f" => {N} = {p} × {q}  [{meta['method']}]")

    print("\n=== SUMMARY ===")
    for N, (p, q, meta) in results.items():
        print(f"{N}: {p} × {q}  [{meta['method']}]")
    return results


if __name__ == "__main__":
    nums = [15, 21, 33, 35, 39, 51]
    run_batch(nums)


Factoring N = 15

 Attempt 1: a = 11 (gcd=1)
  top k=00010000 p=0.2500 → phase=0.062500 → r=15
   invalid r; retrying...

 Attempt 2: a = 5 (gcd=5)
  -> classical gcd factor: 5
 => 15 = 5 × 3  [classical gcd]

Factoring N = 21

 Attempt 1: a = 20 (gcd=1)
  top k=0000100000 p=0.2500 → phase=0.031250 → r=21
   invalid r; retrying...

 Attempt 2: a = 4 (gcd=1)
  top k=0000100000 p=0.1116 → phase=0.031250 → r=21
   invalid r; retrying...

 Attempt 3: a = 15 (gcd=3)
  -> classical gcd factor: 3
 => 21 = 3 × 7  [classical gcd]

Factoring N = 33

 Attempt 1: a = 9 (gcd=3)
  -> classical gcd factor: 3
 => 33 = 3 × 11  [classical gcd]

Factoring N = 35

 Attempt 1: a = 16 (gcd=1)
  top k=000001000000 p=0.1112 → phase=0.015625 → r=35
   invalid r; retrying...

 Attempt 2: a = 27 (gcd=1)
  top k=000001100000 p=0.0625 → phase=0.023438 → r=35
   invalid r; retrying...

 Attempt 3: a = 9 (gcd=1)
  top k=000001000000 p=0.0278 → phase=0.015625 → r=35
   invalid r; retrying...

 Attempt 4: a = 15 (gcd